# Contexte
Le gouvernement péruvien vient d’autoriser les aventuriers en quête de trésors à explorer les 85 182
km² du département de la Madre de Dios. Vous devez réaliser un système permettant de suivre les
déplacements et les collectes de trésors effectuées par les aventuriers. Le gouvernement péruvien
étant très à cheval sur les bonnes pratiques de code, il est important de réaliser un code de qualité,
lisible, et maintenable (oui, ça veut dire avec des tests) !

In [54]:
import re
import sys
import numpy as np

## Lire le fichier d'entrée

In [2]:
def readFile(inputPath, outputPath):
    inputFile = open(inputPath, 'r')
    outputFile = open(outputPath, 'w')
    #outputStrAcommeA = "# {A comme Aventurier} - {Nom de l’aventurier} - {Axe horizontal} - {Axe vertical} - {Orientation} - {Nb. trésors ramassés} \n"
    global nbAdventurers
    global tabAdventurers
    nbCellsWidth = 0
    nbCellsHeight = 0
    global gameMap
    xMontain = None
    yMontain = None
    xTreasure = None
    yTreasure = None
    nbTreasures = None
    
    
    while True:
        # Get next line from file
        line = inputFile.readline()
        if not line:
            break
        
        if re.search("^(A|a).*-", line) == None:
            '''if re.search("^(#).*{(A|a).*comme", line):
                #outputFile.write(outputStrAcommeA)
            elif'''
            if re.search("^(#).*{(T|t).*comme", line):
                commentT = "# {T comme Trésor} - {Axe horizontal} - {Axe vertical} - {Nb. de trésors restants}"
                outputFile.write(commentT)
            else:
                # Carte
                if re.search("^(C|c).*-.*\d", line):
                    nbCellsWidth = int(re.findall("\d+", line)[0])
                    nbCellsHeight = int(re.findall("\d+", line)[1])
                    gameMap = np.empty((nbCellsHeight,nbCellsWidth), list)
                    outputFile.write(line)
                    
                # Montagne    
                if re.search("^(M|m).*-.*\d", line):
                    xMontain = int(re.findall("\d+", line)[0])
                    yMontain = int(re.findall("\d+", line)[1])
                    gameMap[yMontain][xMontain] = "M"
                    outputFile.write(line)
                    
                # Trésor
                if re.search("^(T|t).*-.*\d", line):
                    xTreasure = int(re.findall("\d+", line)[0])
                    yTreasure = int(re.findall("\d+", line)[1])
                    nbTreasures = int(re.findall("\d+", line)[2])
                    gameMap[yTreasure][xTreasure] = ["T", xTreasure, yTreasure, nbTreasures]
                                          
                #print("write line.")   
        else:
            # Adventurer(s)
            nbAdventurers += 1
            listElems = []
            listElems.append("A")
            nameAdv = re.findall("[a-zA-Z]{2,}", line)[0]
            listElems.append(nameAdv)
            columnAdv = int(re.findall("\d+", line)[0])
            listElems.append(columnAdv)
            lineAdv = int(re.findall("\d+", line)[1])
            listElems.append(lineAdv)
            orientationAdv = re.findall("\s+(N|n|S|s|E|e|O|o){1}\s+", line)[0]
            listElems.append(orientationAdv)
            motionSequence = re.findall("[a-zA-Z]{2,}", line)[1]
            listElems.append(motionSequence)
            nbTreasuresFound = 0
            listElems.append(nbTreasuresFound)
            print(listElems)
            indice = nbAdventurers-1
            tabAdventurers.append(listElems)
            gameMap[lineAdv][columnAdv] = listElems
                        
            #txtAname = str(re.findall("^A|a.*-.*-", line)) #+ "{{coordonnées}} {{nb trésors trouvés}}"
            #outputFile.write(txtAname)           
    inputFile.close()
    outputFile.close()

In [3]:
def isInsideMap(y=int,x=int):
    gameMapHeight = gameMap.shape[0]
    gameMapWidth = gameMap.shape[1]
    
    if y < gameMapHeight and y >= 0 and x < gameMapWidth and x >= 0:
        return True
    else:
        return False

In [4]:
def getNewOrientation(orientation,nextMovement):
    if nextMovement != 'A':
        
        if nextMovement == 'D':
            if orientation == 'N':
                orientation = 'E'
            elif orientation == 'S':
                orientation = 'O'
            elif orientation == 'E':
                orientation = 'S'
            elif orientation == 'O':
                orientation = 'N'
        elif nextMovement == 'G':
            if orientation == 'N':
                orientation = 'O'
            elif orientation == 'S':
                orientation = 'E'
            elif orientation == 'E':
                orientation = 'N'
            elif orientation == 'O':
                orientation = 'S'
            
    return orientation

In [5]:
def getNewCoordinates(orientation,y,x):
    newCoordY = y
    newCoordX = x
    
    if orientation == 'S':
        newCoordY = y + 1
    elif orientation == 'E':
        newCoordX = x + 1
    elif orientation == 'N':
        newCoordY = y - 1
    elif orientation == 'O':
        newCoordX = x - 1
    
    return newCoordY, newCoordX

In [6]:
# An adventurer is not in the same cell during >= 2 steps ?
def isNotSameCell(y,x,newY,newX):
    if y==newY and x==newX:
        return False
    else:
        return True

In [68]:
def writeOutputFile(chem):
    gmHeight = gameMap.shape[0]
    gmWidth = gameMap.shape[1]
    outputStrAcommeA = "# {A comme Aventurier} - {Nom de l’aventurier} - {Axe horizontal} - {Axe vertical} - {Orientation} - {Nb. trésors ramassés} \n"
    
    with open(cheminSortie, "a+") as file_output:
        #commentTreasure = "# {T comme Trésor} - {Axe horizontal} - {Axe vertical} - {Nb. de trésors restants}"
        #file_output.write("\n")
        #file_output.write(commentTreasure)
        file_output.write("\n")
        for i in range(gmHeight):
            for j in range(gmWidth):
                if gameMap[i][j]:
                    if gameMap[i][j][0] == 'T':
                        strTreasure = str(gameMap[i][j]).strip('[]')
                        strTreasure = strTreasure.replace(',',' -')
                        strTreasure = strTreasure.replace('\'','')
                        file_output.write(strTreasure)
                        file_output.write("\n")
        
        if len(tabAdventurers) >= 1:
            #commentA = "# {A comme Aventurier} - {Nom de l’aventurier} - {Axe horizontal} - {Axe vertical} - {Orientation} - {Nb. trésors ramassés}"
            file_output.write(outputStrAcommeA)
            print("## Output adventurers ##")
            for i in range(len(tabAdventurers)):
                print(tabAdventurers[i])
                strAdventurer = str(tabAdventurers[i]).strip('[]')
                strAdventurer = strAdventurer.replace(',',' -')
                strAdventurer = strAdventurer.replace('\'','')
                strAdventurer = strAdventurer.replace('-  -','-')
                file_output.write(strAdventurer)
                file_output.write("\n")
            
        file_output.close()

In [77]:
def doSimulation(nbAdventurers, tabAdventurers, gameMap):
    print("Simulation...")
    infLoop = True
    newNbTreasures = 0
    activeTabAdv = []
    for i in range(len(tabAdventurers)):
        activeTabAdv.append(tabAdventurers[i][1])
    
    while infLoop:
        for i in range(len(tabAdventurers)):
            print(tabAdventurers[i])
            print("next move ")
            movementsList = tabAdventurers[i][5]
            if movementsList:
                print(tabAdventurers[i][5])
                peopleName = tabAdventurers[i][1]
                coordX = int(tabAdventurers[i][2])
                coordY = int(tabAdventurers[i][3])
                orientation = tabAdventurers[i][4]
                
                nextMovement = movementsList[0]
            
                if len(movementsList) >= 1:
                    newOrientation = getNewOrientation(orientation,nextMovement)
                    if newOrientation != orientation:
                        tabAdventurers[i][4] = newOrientation
                        
                        # First step'''
                        '''noAdventurers=True
                        cellIsEmpty = True
                        for indice in tabAdventurers:
                            name = tabAdventurers[indice][1]
                            yName = int(tabAdventurers[indice][3])
                            xName = int(tabAdventurers[indice][2])
                            if name != peopleName and yName==coordY and xName==coordX:
                                noAdventurers = False
                        if noAdventurers
                        for i in gameMap:
                            for j in gameMap:
                                if name != peopleName and
                        '''
                        #adventurerInCell = False
                        if gameMap[coordY][coordX] == None:
                            for elem in tabAdventurers:
                                if tabAdventurers[elem][2]==coordX and tabAdventurers[elem][2]==coordY:
                                    #if tabAdventurers[elem][1] == peopleName:
                                        #adventurerInCell = True
                                        gameMap[coordY][coordX] = tabAdventurers[elem]
                        
                        if gameMap[coordY][coordX] and gameMap[coordY][coordX][0] == 'A':
                            if gameMap[coordY][coordX][1] == peopleName:
                                gameMap[coordY][coordX] = tabAdventurers[i]
                                  
                    else:
                        newCoordY, newCoordX = getNewCoordinates(orientation,coordY,coordX)
                        
                        # Cell is inside map
                        if isInsideMap(newCoordY,newCoordX):
                        
                            # Cell is empty
                            if gameMap[newCoordY][newCoordX] == None:
                                
                                if gameMap[coordY][coordX] and gameMap[coordY][coordX][0] != 'T' and gameMap[coordY][coordX][0] != 'M':
                                    gameMap[coordY][coordX] = None
                                tabAdventurers[i][3] = newCoordY
                                tabAdventurers[i][2] = newCoordX
                                # remove movement
                                #tabAdventurers[i][5] = tabAdventurers[i][5][1:]
                                gameMap[newCoordY][newCoordX] = tabAdventurers[i]
                            
                            # Treasure in cell
                            elif gameMap[newCoordY][newCoordX][0] == 'T':
                                
                                # remove 
                                if gameMap[coordY][coordX][0] == 'A' and gameMap[coordY][coordX][1] == peopleName:
                                    gameMap[coordY][coordX] = None
                             
                                nbTreasures = int(gameMap[newCoordY][newCoordX][3])
                                tabAdventurers[i][3] = newCoordY
                                tabAdventurers[i][2] = newCoordX
                                
                                if nbTreasures >= 1:
                                    tabAdventurers[i][6] = int(tabAdventurers[i][6]) + 1
                                    #gameMap[coordY][coordX][3] = int(gameMap[coordY][coordX][3]) - 1
                                    newNbTreasures = nbTreasures - 1
                                    if newNbTreasures == 0:
                                        gameMap[newCoordY][newCoordX] = tabAdventurers[i]
                                    else:
                                        gameMap[newCoordY][newCoordX][3] = newNbTreasures            
                            # Other adventurer or montain in cell : do nothing
                            
                    # remove movement
                    tabAdventurers[i][5] = tabAdventurers[i][5][1:]
            else:
                indice = i
                activeTabAdv[indice] = None
                if all(v is None for v in activeTabAdv):
                    infLoop = False
                    break
           
    # Write output file
    #writeOutputFile()
    return

In [78]:
nbAdventurers = 0
tabAdventurers = []      # informations on adventurers
gameMap = None           # informations on map (size, location of montains, ...)

In [79]:
cheminEntree = './ressources/input04.map'
cheminSortie = './ressources/output04.map'

In [80]:
readFile(cheminEntree, cheminSortie)

['A', 'Lara', 1, 1, 'S', 'AADADAGGA', 0]
output character
['A', 'Paul', 0, 0, 'E', 'AADAGA', 0]
output character
break while true
all files closed
gameMap
.....................
[[list(['A', 'Paul', 0, 0, 'E', 'AADAGA', 0]) 'M' None]
 [None list(['A', 'Lara', 1, 1, 'S', 'AADADAGGA', 0]) 'M']
 [None None None]
 [list(['T', 0, 3, 2]) list(['T', 1, 3, 3]) None]]


In [73]:
print(tabAdventurers)
print("nb Adventurers :", nbAdventurers)
print(gameMap)

[['A', 'Lara', 1, 1, 'S', 'AADADAGGA', 0], ['A', 'Paul', 0, 0, 'E', 'AADAGA', 0]]
nb Adventurers : 2
[[list(['A', 'Paul', 0, 0, 'E', 'AADAGA', 0]) 'M' None]
 [None list(['A', 'Lara', 1, 1, 'S', 'AADADAGGA', 0]) 'M']
 [None None None]
 [list(['T', 0, 3, 2]) list(['T', 1, 3, 3]) None]]


## Exécution de la simulation

In [74]:
doSimulation(nbAdventurers, tabAdventurers, gameMap)

Simulation...
['A', 'Lara', 1, 1, 'S', 'AADADAGGA', 0]
next move 
AADADAGGA
['A', 'Paul', 0, 0, 'E', 'AADAGA', 0]
next move 
AADAGA
['A', 'Lara', 1, 2, 'S', 'ADADAGGA', 0]
next move 
ADADAGGA
['A', 'Paul', 0, 0, 'E', 'ADAGA', 0]
next move 
ADAGA
['A', 'Lara', 1, 3, 'S', 'DADAGGA', 1]
next move 
DADAGGA
['A', 'Paul', 0, 0, 'E', 'DAGA', 0]
next move 
DAGA
['A', 'Lara', 1, 3, 'O', 'ADAGGA', 1]
next move 
ADAGGA
['A', 'Paul', 0, 0, 'S', 'AGA', 0]
next move 
AGA
['A', 'Lara', 0, 3, 'O', 'DAGGA', 2]
next move 
DAGGA
['A', 'Paul', 0, 1, 'S', 'GA', 0]
next move 
GA
['A', 'Lara', 0, 3, 'N', 'AGGA', 2]
next move 
AGGA
['A', 'Paul', 0, 1, 'E', 'A', 0]
next move 
A
['A', 'Lara', 0, 2, 'N', 'GGA', 2]
next move 
GGA
['A', 'Paul', 1, 1, 'E', '', 0]
next move 
['A', 'Lara', 0, 2, 'O', 'GA', 2]
next move 
GA
['A', 'Paul', 1, 1, 'E', '', 0]
next move 
['A', 'Lara', 0, 2, 'S', 'A', 2]
next move 
A
['A', 'Paul', 1, 1, 'E', '', 0]
next move 
['A', 'Lara', 0, 3, 'S', '', 3]
next move 
## Output adventurers 

In [75]:
tabAdventurers

[['A', 'Lara', 0, 3, 'S', '', 3], ['A', 'Paul', 1, 1, 'E', '', 0]]

In [76]:
gameMap

array([[None, 'M', None],
       [None, list(['A', 'Paul', 1, 1, 'E', '', 0]), 'M'],
       [None, None, None],
       [list(['A', 'Lara', 0, 3, 'S', '', 3]), list(['T', 1, 3, 2]),
        None]], dtype=object)

In [99]:
gameMap.shape

(4, 3)

## Tests

### Le format des fichiers en entrée

In [58]:
def isCorrectInputFormat(fileInputPath):
    inputFile = open(fileInputPath, 'r')
    carte = False
    nbCarte = 0
    montagne = False
    nbMontagne = 0
    tabMontagne = []
    tresor = False
    nbTresor = 0
    tabTresor = []
    adventurers = False
    nbAdventurers = 0
    adventurersTabCoord = []
    while True:
        line = inputFile.readline()
        if line:
            if not carte and re.search("^(C|c){1}\s+-{1}\s+\d+\s+-{1}\s+\d+", line):
                #return True
                carte = True
                nbCarte += 1
            elif carte and re.search("^(M|m){1}\s+-{1}\s+\d+\s+-{1}\s+\d+", line):
                montagne = True
                nbMontagne += 1
                xMontain = int(re.findall("\d+", line)[0])
                yMontain = int(re.findall("\d+", line)[1])
                tabMontagne.append([yMontain,xMontain])
            elif carte and re.search("^(T|t){1}\s+-{1}\s+\d+\s+-{1}\s+\d+", line):
                tresor = True
                nbTresor += 1
                xTresor = int(re.findall("\d+", line)[0])
                yTresor = int(re.findall("\d+", line)[1])
                tabTresor.append([yTresor, xTresor])
            elif carte and re.search("^(A|a).*-", line):
                nameAdv = re.findall("[a-zA-Z]{2,}", line)[0]
                xAdv = int(re.findall("\d+", line)[0])
                yAdv = int(re.findall("\d+", line)[1])
                adventurers = True
                nbAdventurers += 1
                adventurersTabCoord.append([nameAdv, yAdv, xAdv])
        else:
            #return False 
            break
    if nbCarte!= 1:
        return False
    
    for i in range(len(tabMontagne)):
        y1 = tabMontagne[i][0]
        x1 = tabMontagne[i][1]
        for j in range(i+1,len(tabMontagne)):
            y2 = tabMontagne[j][0]
            x2 = tabMontagne[j][1]
            if y1 == y2 and x1 == x2:
                sys.stderr.write("2 montains in the same cell !\n")
                return False
    
    for i in range(len(tabTresor)):
        y1 = tabTresor[i][0]
        x1 = tabTresor[i][1]
        for j in range(i+1,len(tabTresor)):
            y2 = tabTresor[j][0]
            x2 = tabTresor[j][1]
            if y1 == y2 and x1 == x2:
                sys.stderr.write("2 differents treasures in the same cell !\n")
                return False
            
    for i in range(len(adventurersTabCoord)):
        y1 = adventurersTabCoord[i][1]
        x1 = adventurersTabCoord[i][2]
        for j in range(i+1,len(adventurersTabCoord)):
            y2 = adventurersTabCoord[j][1]
            x2 = adventurersTabCoord[j][2]
            if y1 == y2 and x1 == x2:
                sys.stderr.write("2 differents adventurers in the same cell !\n")
                return False
            
    return True

In [59]:
isCorrectInputFormat('ressources/inputIncorrectFormat.map')

False

In [60]:
isCorrectInputFormat('ressources/input2playersInSameCell.map')

2 differents adventurers in the same cell !


False

In [53]:
isCorrectInputFormat(cheminEntree)

True

### Le format des fichiers en sortie

In [204]:
def isCorrectOutputFormat(fileOutputPath):
    formatC = None
    formatM = None
    formatT = None
    formatA = None
    outputFile = open(fileOutputPath, 'r')
    while True:
        line = outputFile.readline()
        if line:
            if re.match("^(C|c){1}", line): 
                if re.match("^(C|c){1}\s+-{1}\s+\d+\s+-{1}\s+\d+", line):
                    formatC = True
                else:
                    return False
            elif re.match("^(M|m){1}", line):
                if re.match("^(M|m){1}\s+-{1}\s+\d+\s+-{1}\s+\d+", line):
                    formatM = True
                else:
                    return False
            elif re.match("^(T|t){1}", line):
                if re.match("^(T|t){1}\s+-{1}\s+\d+\s+-{1}\s+\d+\s+-{1}\s+\d+", line):
                    formatT = True
                else:
                    return False
            elif re.match("^(A|a){1}", line):
                if re.match("^(A|a){1}\s+-{1}\s+[A-Za-z]+\s+-{1}\s+\d+\s+-{1}\s+\d+\s+-{1}\s+(N|S|E|O)\s+-{1}\s+[\d]+", line):
                    formatT = True
                else:
                    return False
            elif re.match("^(#)+", line) or re.search("^(\n)+", line):
                continue
            else:
                print("incorrect char !!!")
                print(line)
                return False
        else:
            break
    if formatC==True and formatM!=False and formatT!=False and formatA!=False:
        return True
    else:
        return False

In [203]:
isCorrectOutputFormat('ressources/output02.map')

incorrect char !!!
!


False

In [197]:
isCorrectOutputFormat(cheminSortie)

True